In [ ]:
import tensorflow as tf

import sys
sys.path.insert(0, '../fairml')
import plot
import generate
import models
import environment


import itertools
from importlib import reload
reload(generate)
reload(plot)


generate_toys = generate.toys_simple
#generate_toys = generate.toys_expo_Z
#generate_toys = generate.toys_discrete_Z
#generate_toys = generate.toys_peaked_Z

plot.show_variates(generate_toys, 10000)

# Train a single environment as a test

In [ ]:
import uuid
reload(generate)
reload(plot)
reload(models)
reload(environment)

clf_settings = {'depth':2, 'width':20}
adv_settings = {'adv_type':'GMM'}
opt_settings = {'lam':1, 'opt_type':'AdamOptimizer', 'learning_rate':0.01, 'projection':False}
trn_settings = {'n_pretrain':30, 'batch_size':1000, 'n_train':100}

# make the environment
tfe = environment.TFEnvironment(generate_toys, 'env_{}'.format(uuid.uuid4()))
tfe.build_graph(clf_settings, adv_settings)
tfe.build_loss()
tfe.build_opt(**opt_settings)
tfe.initialise_variables()

# pretrain
batch_size = trn_settings['batch_size']
n_pretrain = trn_settings['n_pretrain']
for _ in range(n_pretrain):
    tfe.pretrain_step_clf(batch_size, write=False)
for _ in range(n_pretrain):
    tfe.train_step_adv(batch_size, write=False)

# train
n_train = trn_settings['n_train']
for _ in range(n_train):

    tfe.train_step_clf(batch_size)
    for __ in range(5):
        tfe.train_step_adv(batch_size)

print('Done')

In [ ]:
reload(plot)
tfe.show_performance(4000)
tfe.show_history()

# Train multiple environments

In [ ]:
import itertools
from importlib import reload
reload(environment)
reload(models)

# setting signature
clf_settings = {'depth':2, 'width':20}
adv_settings = {'adv_type':'MINE'}
opt_settings = {'lam':1, 'opt_type':'AdamOptimizer', 'learning_rate':0.01, 'projection':False}
trn_settings = {'n_pretrain':30, 'batch_size':1000, 'n_train':100}

# prepare the loop
adversaries = ['MINE', 'GMM', 'PtEst']
projections = [True, False]
optimizers = ['AdamOptimizer']

# prepare the results
envs = {adv:{proj:{opt:[] for opt in optimizers} for proj in projections} for adv in adversaries}

# run the loop
for adv, proj, opt in itertools.product(adversaries, projections, optimizers):
    
    print(adv, proj, opt)

    adv_settings['adv_type'] = adv
    opt_settings['projection'] = proj
    opt_settings['opt_type'] = opt
    envs[adv][proj][opt] = environment.bootcamp(10, generate_toys, clf_settings, adv_settings, opt_settings, trn_settings)

print('Done')

# Compare the performance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from importlib import reload
reload(plot)

fig, ax = plt.subplots(6, figsize=(10,15))
fig.suptitle('Performance for different adversaries (lambda={})'.format(opt_settings['lam']))
colors ={'MINE':'navy', 'GMM':'royalblue', 'PtEst':'tomato'}
for adv, opt in itertools.product(adversaries, optimizers):
    
    # classifier loss
    plot.history(ax[0], [e.history['L_clf'] for e in envs[adv][False][opt]], color=colors[adv], label='Classifier loss ({})'.format(adv))

    # adversary loss
    plot.history(ax[1], [e.history['L_adv'] for e in envs[adv][False][opt]], color=colors[adv], label='Adversary loss ({})'.format(adv))
                 
    # combined loss
    plot.history(ax[2], [e.history['L_comb'] for e in envs[adv][False][opt]], color=colors[adv], label='Combined loss ({})'.format(adv))
    
    # AUROC
    plot.history(ax[3], [e.history['auroc-mean'] for e in envs[adv][False][opt]], color=colors[adv], label='mean(AUROC), Z=(1,0,-1) ({})'.format(adv))
    plot.history(ax[4], [e.history['auroc-std'] for e in envs[adv][False][opt]], color=colors[adv], label='std(AUROC), Z=(1,0,-1) ({})'.format(adv))
    
    # KS
    plot.history(ax[5], [e.history['KS'] for e in envs[adv][False][opt]], color=colors[adv], label='KS metric ({})'.format(adv))



for i in range(6):
    ax[i].legend(loc='best')

fig.show()




In [ ]:

for adv, opt in itertools.product(adversaries, optimizers):
    
    # create the figure
    fig, ax = plt.subplots(6, figsize=(10,15))
    fig.suptitle('Performance for {} with {} (lambda={})'.format(adv, opt, opt_settings['lam']))
    
    # classifier loss
    plot.history(ax[0], [e.history['L_clf'] for e in envs[adv][True][opt]], color='crimson', label='Classifier loss (with proj)')
    plot.history(ax[0], [e.history['L_clf'] for e in envs[adv][False][opt]], color='olive', label='Classifier loss (w/o proj)')

    # adversary loss
    plot.history(ax[1], [e.history['L_adv'] for e in envs[adv][True][opt]], color='crimson', label='Adversary loss (with proj)')
    plot.history(ax[1], [e.history['L_adv'] for e in envs[adv][False][opt]], color='olive', label='Adversary loss (w/o proj)')

    # combined loss
    plot.history(ax[2], [e.history['L_comb'] for e in envs[adv][True][opt]], color='crimson', label='Combined loss (with proj)')
    plot.history(ax[2], [e.history['L_comb'] for e in envs[adv][False][opt]], color='olive', label='Combined loss (w/o proj)')

    # AUROC
    plot.history(ax[3], [e.history['auroc-mean'] for e in envs[adv][True][opt]], color='crimson', label='mean(AUROC) (with proj)')
    plot.history(ax[3], [e.history['auroc-mean'] for e in envs[adv][False][opt]], color='olive', label='mean(AUROC) (w/o proj)')
    
    plot.history(ax[4], [e.history['auroc-std'] for e in envs[adv][True][opt]], color='crimson', label='std(AUROC) (with proj)')
    plot.history(ax[4], [e.history['auroc-std'] for e in envs[adv][False][opt]], color='olive', label='std(AUROC) (w/o proj)')
    
    # KS
    plot.history(ax[5], [e.history['KS'] for e in envs[adv][True][opt]], color='crimson', label='KS metric (with proj)')
    plot.history(ax[5], [e.history['KS'] for e in envs[adv][False][opt]], color='olive', label='KS metric (w/o proj)')



    
    for i in range(6):
        ax[i].legend(loc='best')
        